IMPORT LIBRARIES

In [2]:
import numpy as np 
import pandas as pd 

READ THE DATA:

- In this challenge you will use the dataset Titanic disaster.
- With the following lines of code, you can import the dataset form the file "train.csv", extract te column of labels (column "Survived") and read the infos of the other features.

---



In [4]:
from google.colab import drive
#if you downloaded the file in the Colab directory in your Google Drive, use the following lines of code
drive.mount('/gdrive', force_remount=True)
data_file="/gdrive/My Drive/Colab Notebooks/train.csv" 
dataset=pd.read_csv(data_file)
features=dataset.drop('Survived',axis=1)
labels=dataset.Survived
features.info()
labels

Mounted at /gdrive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Name         891 non-null    object 
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        204 non-null    object 
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB


0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

FEATURES ENGINEERING:

- Evaluate if there are features with missing values and modify them (see https://scikit-learn.org/stable/modules/impute.html)
- Find meaningful features (some features are not needed because do not contain useful infos)
- Encode categorical features with numerical values (see https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features)

In [ ]:
#Write your code here for feature engineering



TRAIN AND TEST DIFFERENT CLASSIFIERS:

- Find the best parameters for each classifier with a stratified 10-fold cross validation (see https://scikit-learn.org/stable/modules/cross_validation.html)

- Try these models:

>- Naive Bayes (see https://scikit-learn.org/stable/modules/naive_bayes.html) 
>- kNN (see https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification)
>- Decision tree (see https://scikit-learn.org/stable/modules/tree.html)
>- Logistic regression (see https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)

- Show the final performances in terms of accuracy 

In [ ]:
#Write your code here for applying cross validation to each model and optimize the classifiers' parameters

